In [1]:
import os
import numpy as np
from matplotlib.lines import Line2D
from scipy.stats import pearsonr, gaussian_kde, linregress, ttest_ind, sem, zscore
from sklearn.metrics import r2_score, mean_squared_error
from scipy.stats import norm
from scipy.stats import percentileofscore
from sklearn.utils.validation import check_random_state
from math import factorial
from more_itertools import distinct_permutations
import statsmodels.api as sm

import matplotlib.pyplot as plt

import pandas as pd
from sklearn.model_selection import KFold, train_test_split, StratifiedKFold
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
from sklearn.linear_model import LinearRegression
import statsmodels.api as sm
from scipy import stats

import numpy as np
from scipy.stats import linregress
from scipy.stats import pearsonr
import matplotlib.pyplot as plt

from sklearn.metrics import r2_score, mean_squared_error
from sklearn.model_selection import KFold, ParameterGrid, train_test_split
from tqdm.notebook import tqdm

import warnings
import random
#from torch.utils.data import SubsetRandomSampler
from sklearn.utils import resample

from sklearn.preprocessing import MinMaxScaler
from statsmodels.stats.diagnostic import linear_harvey_collier
from scipy.stats import chi2
from sklearn.model_selection import train_test_split
from colorama import Fore, Style, init
from sklearn.metrics import roc_auc_score
from collections import Counter

from statsmodels.stats.diagnostic import linear_harvey_collier
from scipy.stats import chi2
from sklearn.model_selection import train_test_split
from colorama import Fore, Style, init
from sklearn.metrics import roc_auc_score

warnings.filterwarnings("ignore")



In [2]:
from scipy.stats import chi2

def summarize_feature(df, feature, df_out, stat_cols=(10, 50), df_offset=2):

    d = df[df["Feature"] == feature]
    if d.empty:
        return df_out  

    for k in stat_cols:
        col = f"hl_statistic_{k}"
        if col in d.columns:
            df_chi = k - df_offset
            combined_p = 1 - chi2.cdf(d[col].mean(), df_chi)
            df_out.loc[feature, f"hl_pvalue_combined_{k}"] = combined_p

    # --- OR summary ---
    if "OR" in d.columns:
        or_mean = d["OR"].mean()
        or_std = d["OR"].std(ddof=1)
        df_out.loc[feature, "OR"] = or_mean
        df_out.loc[feature, "2.5%"] = or_mean - or_std
        df_out.loc[feature, "97.5%"] = or_mean + or_std

    # --- min/max de los IC originales (tal como lo estabas haciendo) ---
    if "2.5%" in d.columns:
        df_out.loc[feature, "minOR"] = d["2.5%"].min()
    if "97.5%" in d.columns:
        df_out.loc[feature, "maxOR"] = d["97.5%"].max()

    return df_out


features = ["Mono", "One", "Two", "Three", "Total"] 




def hosmer_lemeshow(y_true, y_prob, g):
    hl_df = pd.DataFrame({
        "observed": y_true,
        "predicted_probability": y_prob
    }).dropna()


    hl_df["group"] = pd.qcut(hl_df["predicted_probability"], g, duplicates="drop")

    hl_table = hl_df.groupby("group").apply(
        lambda x: pd.Series({
            "observed": x["observed"].sum(),
            "expected": x["predicted_probability"].sum(),
            "total": len(x)
        })
    )

    hl_table["observed_neg"] = hl_table["total"] - hl_table["observed"]
    hl_table["expected_neg"] = hl_table["total"] - hl_table["expected"]


    hl_statistic = (
        ((hl_table["observed"] - hl_table["expected"])**2) / hl_table["expected"] +
        ((hl_table["observed_neg"] - hl_table["expected_neg"])**2) / hl_table["expected_neg"]
    ).sum()


    dof = hl_table.shape[0] - 2
    p_value = 1 - chi2.cdf(hl_statistic, dof)

    return hl_statistic, p_value



def summarize_feature_by_covar(df, covar, feature, df_out, stat_cols=(10, 50), df_offset=2):
    d = df[(df["Covar"] == covar) & (df["Feature"] == feature)]
    if d.empty:
        return df_out

    idx = (covar, feature)

    for k in stat_cols:
        col = f"hl_statistic_{k}"
        if col in d.columns:
            df_chi = k - df_offset
            df_out.loc[idx, f"hl_pvalue_combined_{k}"] = 1 - chi2.cdf(d[col].mean(), df_chi)

    or_mean = d["OR"].mean()
    or_std  = d["OR"].std(ddof=1)
    df_out.loc[idx, "OR"] = or_mean
    df_out.loc[idx, "2.5%"] = or_mean - or_std
    df_out.loc[idx, "97.5%"] = or_mean + or_std

    df_out.loc[idx, "minOR"] = d["2.5%"].min()
    df_out.loc[idx, "maxOR"] = d["97.5%"].max()

    df_out.loc[idx, "n_iter"] = d.shape[0]

    return df_out


## Load data

In [3]:
import time

t0 = time.perf_counter()






In [4]:
data = pd.read_parquet('data/BBAG-cross.parquet')

In [5]:
data = data[data.country != 'Slovakia'].reset_index(drop =  True)

In [6]:
Counter(data.country)

Counter({'Austria': 4320,
         'Belgium': 4942,
         'Czechia': 5575,
         'Switzerland': 2634,
         'Germany': 4567,
         'Denmark': 2609,
         'Estonia': 7522,
         'Spain': 4912,
         'France': 3640,
         'Hungary': 3010,
         'Italy': 4541,
         'Netherlands': 2546,
         'Poland': 3612,
         'Portugal': 1914,
         'Sweden': 2792,
         'Slovenia': 5291,
         'Luxembourg': 2104,
         'Greece': 2599,
         'Croatia': 2798,
         'Bulgaria': 1953,
         'Cyprus': 1205,
         'Finland': 1982,
         'Lithuania': 2033,
         'Latvia': 1684,
         'Malta': 1259,
         'Romania': 2083})

In [7]:
vars_ = ['Mono', 'One',	'Two',	'Three', 'Total']


In [8]:
data[vars_] = data[vars_] / 100

# Odds ratios

## Without co-vars

### Without iterations

In [10]:
results_merge_df_all = data.copy()

results_merge_df_all = results_merge_df_all.loc[:, ~results_merge_df_all.columns.duplicated()]

df_directions_odd = pd.DataFrame()

for i in vars_:
    
    y_ols = results_merge_df_all['GAP_bin']

    
    X_ols = results_merge_df_all[[i]]
    X_train, X_test, y_train, y_test = train_test_split(X_ols, y_ols, test_size=0.2, random_state=42)
    
    
    X_train['intercept'] = 1
    X_test['intercept'] = 1
    
    model = sm.Logit(y_train, X_train).fit(disp = 0)
    
    params = model.params
    conf = np.exp(model.conf_int())
    conf['OR'] = np.exp(params)
    conf['z'] =model.tvalues
    conf['P>|z|'] =model.pvalues
    conf.columns = ['2.5%', '97.5%', 'OR', 'z', 'P>|z|']
    
    df = conf.loc[i:i]
    df['Feature'] = i
    df_directions_odd = pd.concat([df_directions_odd, df])

df_directions_odd = df_directions_odd.reset_index(drop=True)

df_directions_odd.to_excel('Results/cross/cross_OR_-removing-slovakia-proportions.xlsx')
df_directions_odd

,2.5%,97.5%,OR,z,P>|z|,Feature
0,2.800749,3.371425,3.072868,23.729241,1.800084e-124,Mono
1,0.544988,0.710626,0.622320,-7.005826,2.455334e-12,One
2,0.213613,0.294440,0.250791,-16.894970,4.899618e-64,Two
3,0.506979,0.627614,0.564081,-10.514530,7.404916e-26,Three
4,0.296887,0.357364,0.325725,-23.716052,2.462736e-124,Total


### 1000-iteration

In [12]:


count = 0

results_dict = {}
for i in vars_:
    results_dict[i + "_y"] = pd.Series(dtype=float)
    results_dict[i + "_ypred"] = pd.Series(dtype=float)

results_merge_df_all = data.copy()
results_merge_df_all.dropna(subset=vars_ + ["GAP_bin"], inplace=True)
results_merge_df_all.reset_index(inplace=True, drop=True)

df_directions_odd = pd.DataFrame()

n_boosts = 10  
test_size = 500 / results_merge_df_all.shape[0]

for boosts in range(n_boosts):
    for i in vars_:

        y_ols = results_merge_df_all["GAP_bin"]
        X_ols = results_merge_df_all[[i]]

        X_train, X_test, y_train, y_test = train_test_split(
            X_ols, y_ols,
            test_size=test_size,
            stratify=results_merge_df_all["GAP_bin"],
            random_state=boosts
        )

        
        X_train["intercept"] = 1
        X_test["intercept"] = 1


        model = sm.Logit(y_train, X_train).fit(disp=0)


        y_test_pred = model.predict(X_test)

        results_dict[i + "_y"] = pd.concat([results_dict[i + "_y"], y_test], axis=0)
        results_dict[i + "_ypred"] = pd.concat([results_dict[i + "_ypred"], y_test_pred], axis=0)

        params = model.params
        conf = np.exp(model.conf_int())
        conf["OR"] = np.exp(params)
        conf["z"] = model.tvalues
        conf["P>|z|"] = model.pvalues
        conf.columns = ["2.5%", "97.5%", "OR", "z", "P>|z|"]

        df = conf.loc[i:i].copy()
        df["Feature"] = i

        hl10_stat, hl10_p = hosmer_lemeshow(y_test, y_test_pred, g=10)
        hl50_stat, hl50_p = hosmer_lemeshow(y_test, y_test_pred, g=50)
        hl100_stat, hl100_p = hosmer_lemeshow(y_test, y_test_pred, g=100)

        df["hl_statistic_10"] = hl10_stat
        df["hl_pvalue_10"] = hl10_p

        df["hl_statistic_50"] = hl50_stat
        df["hl_pvalue_50"] = hl50_p

        df["hl_statistic_100"] = hl100_stat
        df["hl_pvalue_100"] = hl100_p

        df_directions_odd = pd.concat([df_directions_odd, df], axis=0)

    df_directions_odd = df_directions_odd.reset_index(drop=True)

df_directions_odd;


In [13]:
df_directions_odd_f = pd.DataFrame()

for f in vars_:
    df_directions_odd_f = summarize_feature(df_directions_odd, f, df_directions_odd_f, stat_cols=(10, 50, 100))

df_directions_odd_f.to_excel('Results/cross/cross_OR_-removing-slovakia-proportions-1000-iter.xlsx')
df_directions_odd_f

,hl_pvalue_combined_10,hl_pvalue_combined_50,hl_pvalue_combined_100,OR,2.5%,97.5%,minOR,maxOR
Mono,0.297950,0.400736,0.993578,3.096315,3.090437,3.102192,2.838623,3.373420
One,0.144573,0.298007,0.988340,0.622593,0.619750,0.625435,0.549032,0.708016
Two,0.158351,0.392823,0.996717,0.259275,0.258038,0.260512,0.223107,0.301160
Three,0.097559,0.460489,0.994873,0.550249,0.548614,0.551883,0.496886,0.607950
Total,0.304208,0.380079,0.994973,0.323209,0.322595,0.323824,0.296661,0.352540


In [11]:
dt = time.perf_counter() - t0
m, s = divmod(dt, 60)
print(f"Tiempo total: {int(m)} min {s:.1f} s")

Tiempo total: 0 min 14.7 s
